In [20]:
import requests
import json
import pandas as pd

import os
eia_api_key = os.environ['EIA_API_KEY']

# Fetching Raw Data

In [74]:
## GENERATION

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"respondent": ["CISO"]},
        'start': '2021-06-01',
        "data": ["value"],
        'end':'2021-06-01T23',
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/",
    params=params, 
    headers=headers
)
pd.DataFrame(response.json()['response']['data']).to_csv('raw/generation.csv', index=False)

In [75]:
# Capacity
headers = {"X-Params": json.dumps({
    "frequency": "monthly",
    "data": [
        "nameplate-capacity-mw"
    ],
    "facets": {
        "balancing_authority_code": [
            "CISO"
        ]
    },
    "start": "2021-06",
    "end": "2021-06",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/",
    params=params, 
    headers=headers
)
response.json()
pd.DataFrame(response.json()['response']['data']).to_csv('raw/capacity.csv', index=False)

In [85]:
## Imports/Exports

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"fromba": ["CISO"]},
        'start': '2021-06-01',
        "data": ["value"],
        'end':'2021-06-01T23',
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/interchange-data/data/",
    params=params, 
    headers=headers
)
pd.DataFrame(response.json()['response']['data']).to_csv('raw/imports_exports.csv', index=False)

In [129]:
## Load Forecast. 

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"respondent": ["CISO"]},
        'start': '2022-06-01T00',
        'end':'2022-06-01T23',
        "data": ["value"],
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}

response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/region-data/data/",
    params=params, 
    headers=headers
)

pd.DataFrame(response.json()['response']['data']).to_csv('raw/load_forecast.csv', index=False)

# Transforming Data

In [69]:
# Generation

# Time is in UTC
df = pd.read_csv('raw/generation.csv')
df = pd.pivot_table(
    df,
    values="value",
    index=["respondent", "period"],
    columns="type-name"
).fillna(0).reset_index()

# Rename and reformat to align with ENTSO-E
df['datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df.drop(['period'], inplace=True, axis=1)
df = df.rename(columns={
    "respondent":"Balancing Region",
    "Coal": "Coal (MW)",
    "Hydro": "Hydro (MW)",
    "Natural gas": "Natural Gas (MW)",
    "Hydro": "Hydro (MW)",
    "Nuclear": "Nuclear (MW)",
    "Other": "Other (MW)",
    "Petroleum": "Petroleum (MW)",
    "Solar": "Solar (MW)",
    "Wind": "Wind (MW)"
}
)
df.to_csv('generation.csv', index=False)

In [101]:
# Capacity
df = pd.read_csv('raw/capacity.csv')
df = df.groupby(["period","energy-source-desc","balancing_authority_code"]).agg({"nameplate-capacity-mw":"sum"})
df.reset_index().rename(columns={
    "period":"datetime",
    "nameplate-capacity-mw":"Capacity (MW)",
    "energy-source-desc":"Fuel Type"
}).to_csv('capacity.csv', index=False)

In [102]:
# Imports/Exports
df = pd.read_csv('raw/imports_exports.csv')
df['datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df = df.groupby(['fromba','datetime']).agg({'value': 'sum'})
df['import (MW)'] = df.value.apply(lambda x: min(x,0))
df['export (MW)'] = df.value.apply(lambda x: max(x,0))
df.drop(['value'], inplace=True, axis=1)
df.reset_index().rename(columns={"fromba":"Balancing Region"}).to_csv('imports_exports.csv', index=False)

In [138]:
# Load Forecast
df = pd.read_csv('raw/load_forecast.csv')
df = df[df['type'].isin(['D','DF'])]
df['datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df.drop(['period'], inplace=True, axis=1)
df = pd.pivot_table(
    df,
    values="value",
    index=["respondent", "datetime"],
    columns="type-name"
).reset_index()
df.rename(columns={
    "Day-ahead demand forecast": "Demand Forecast (MW)",
    "Demand": "Actual Total Load (MW)",
    "respondent": "Balancing Region",
}).to_csv('load_forecast.csv',index=False)